In [1]:
%run "../Libraries/ADP_Spain_MDM_Process"

In [2]:
process = "MDM_SP"
logger_name = "MDM_SP_Data_TobeProcessed"
level_action = "DEFAULT"
log_level = "DEFAULT"

In [3]:
#################################################################################
""" Main Process for Farmatic Master Data Ingest
    Process Master Data Files,Validates Structure and Distributes to the Canonical Folder 

"""
 #Who                 When           What
 #Victor Salesa       15/10/2018     Initial Version
 #Victor Salesa       05/11/2018     Changed pharmacies file load
 #Ana Perez           25/03/2019     Included log managment and exception managment
 #                                   Call the four master data functions
 #Ana Perez           06/05/2019     Changed NoteBookName and logger_name 
 #Ana Perez           07/05/2019     Generate row in CTL.JOB table
 #Ana Perez           08/05/2019     Generate a row in CTL.JOB table by master data file
################################################################################

try:
  #Create parameter to launch task from notebook launcher
  try:
    level_action = dbutils.widgets.get("level_action") 
    log_level = dbutils.widgets.get("log_level")  
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)   
  except Exception as err:
    ADP_log_info(process, logger_name, "DEFAULT", "DEFAULT", "Internal Call parameter was not Created Please Re-run", sys._getframe().f_code.co_name)   

    dbutils.widgets.text("level_action", "DEFAULT","Level Action")
    dbutils.widgets.text("log_level","DEFAULT","Log Level")

    level_action = dbutils.widgets.get("level_action") 
    log_level = dbutils.widgets.get("log_level")

  #end exception 
  
  process_master = process + '_PRODUCT'
  job_id = str(StartADPProcess(process_master)) 
  product_ok = ValidateAndDistributeSpainProductMasterData()
  EndADPProcess(product_ok,job_id=int(job_id))
  
  process_master = process + '_PRODUCT_CAT'
  job_id = str(StartADPProcess(process_master))
  categories_ok = ValidateAndDistributeSpainCategoriesMasterData()
  EndADPProcess(categories_ok,job_id=int(job_id))

  process_master = process + '_PHARMACY'
  job_id = str(StartADPProcess(process_master))
  pharmacies_ok = ValidateAndDistributeSpainPharmaciesMasterData()
  EndADPProcess(pharmacies_ok,job_id=int(job_id))

  process_master = process + '_MANUFACTURER'
  job_id = str(StartADPProcess(process_master))
  manufacturers_ok = ValidateAndDistributeSpainManufacturersMasterData()
  EndADPProcess(manufacturers_ok,job_id=int(job_id))
 

  ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name) 

except Exception as err:
  message = ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
  message = str(message).replace("'","").replace('"','').replace('\\n','\n')
  
  #Update Job Id result in JOB TABLE
  EndADPProcess(0,job_id=int(job_id),log_error_code=50003,log_error_message=message)
  raise Exception(err)
